In [8]:
from hiive.mdptoolbox import mdp
import mdptoolbox
import gym
import numpy as np
import sys
import os
np.random.seed(44)

In [44]:
# env = gym.make('FrozenLake-v0', map_name="8x8").env
env = gym.make("Taxi-v3")
P = np.zeros((env.action_space.n, env.observation_space.n, env.observation_space.n))
R = np.zeros((env.action_space.n, env.observation_space.n, env.observation_space.n))


In [45]:
for state in env.P:
    for action in env.P[state]:
        for prob, new_state, reward, done in env.P[state][action]:
            P[action][state][new_state] += prob
            R[action][state][new_state] = reward

In [46]:
vi = mdptoolbox.mdp.ValueIteration(P, R, discount=0.9, epsilon=1e-12, max_iter=1e12)
vi.run()
policy = vi.policy # result is (0, 0, 0)
vi.iter

313

In [47]:
pi = mdptoolbox.mdp.PolicyIteration(P, R, discount=0.9)
pi.run()
pi_policy = pi.policy
pi.iter

16

In [48]:
q = mdptoolbox.mdp.QLearning(P, R, discount=0.9, n_iter=1e5)
q.run()
q_policy = q.policy

## Single Step Run

In [52]:
current_state = env.reset()
rewards = []

In [ ]:

act = int(policy[current_state])

print(act)
new_state, reward, finished, _ = env.step(act)
rewards.append(reward)
current_state = new_state
print(finished, reward)
env.render()

In [ ]:

act = int(pi_policy[current_state])

print(act)
new_state, reward, finished, _ = env.step(act)
rewards.append(reward)
current_state = new_state
print(finished, reward)
env.render()

In [84]:

act = int(q_policy[current_state])

print(act)
new_state, reward, finished, _ = env.step(act)
rewards.append(reward)
current_state = new_state
print(finished, reward)
env.render()

1
False -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


## Multiple Step Run

In [86]:
done = False
ep = 0
current_state = env.reset()
while not done:
    ep += 1
    act = int(policy[current_state])
    new_state, reward, done, _ = env.step(act)
    current_state = new_state
#     print(done, reward, new_state)
print("Finished at step: ", ep)
env.render()    

Finished at step:  14
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [87]:
done = False
ep = 0
current_state = env.reset()
while not done:
    ep += 1
    act = int(pi_policy[current_state])
    new_state, reward, done, _ = env.step(act)
    current_state = new_state
#     print(done, reward, new_state)
print("Finished at step: ", ep)
env.render()    

Finished at step:  15
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [88]:
done = False
ep = 0
current_state = env.reset()
while not done:
    ep += 1
    act = int(q_policy[current_state])
    new_state, reward, done, _ = env.step(act)
    current_state = new_state
#     print(done, reward, new_state)
print("Finished at step: ", ep)
env.render()    

Finished at step:  200
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
